In [1]:

import os
import torch
from ultralytics import YOLO
from roboflow import Roboflow

# Check available GPUs
num_gpus = torch.cuda.device_count()
device_ids = list(range(num_gpus))  # Example: [0,1,2,3] if 4 GPUs are available

# Define dataset path (update if needed)
dataset_path = "/home/mansi/Sumit Kumar, Satyendra/Eyes Final fgs/aptos org"
data_yaml = os.path.join(dataset_path, "data.yaml")

# Check if dataset exists
if not os.path.exists(data_yaml):
    print("Dataset not found! Downloading...")
    rf = Roboflow(api_key="50TWiA9mukNXx9kJkYuO")
    project = rf.workspace("eyes-final").project("aptos-n5h2u")
    version = project.version(4)
    dataset = version.download("yolov8")
    os.rename(dataset.location, dataset_path)
else:
    print("Dataset already exists. Using the existing dataset.")

# Load YOLO model
model = YOLO("yolov8n")  # Adjust model if needed




Dataset not found! Downloading...
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Aptos-4 in yolov8:: 100%|███████████████████████████████████████████████████████████| 7080/7080 [00:03<00:00, 2197.65it/s]


In [2]:
import os
from collections import defaultdict

# Base dataset path
dataset_path = "/home/mansi/Sumit Kumar, Satyendra/Eyes Final fgs/aptos org"

# Corrected label paths
labels_dirs = [
    os.path.join(dataset_path, "val", "labels"),
    os.path.join(dataset_path, "test", "labels"),
]

# Dictionary to track images per class
class_to_images = defaultdict(set)

# Iterate and count
for labels_dir in labels_dirs:
    if not os.path.exists(labels_dir):
        print(f"Missing directory: {labels_dir}")
        continue

    print(f"Scanning: {labels_dir}")
    for label_file in os.listdir(labels_dir):
        if not label_file.endswith(".txt"):
            continue

        label_path = os.path.join(labels_dir, label_file)

        with open(label_path, "r") as f:
            lines = f.readlines()
            for line in lines:
                if line.strip() == "":
                    continue
                class_id = int(line.split()[0])
                class_to_images[class_id].add(label_file)

# Output counts
print("\nNumber of images per class:")
for class_id in range(5):  # Classes 0 to 4
    count = len(class_to_images[class_id])
    print(f"Class {class_id}: {count} images")


Missing directory: /home/mansi/Sumit Kumar, Satyendra/Eyes Final fgs/aptos org/val/labels
Scanning: /home/mansi/Sumit Kumar, Satyendra/Eyes Final fgs/aptos org/test/labels

Number of images per class:
Class 0: 91 images
Class 1: 19 images
Class 2: 40 images
Class 3: 9 images
Class 4: 19 images


In [3]:
# Train with optimized settings
model.train(
    data=data_yaml,
    name="aptos org",
    device=device_ids,  # Use all available GPUs
    epochs=100, 
    batch=64,  # Adjust batch size for better GPU utilization
    workers=16,  # Use multiple data loading workers
#     optimizer="SGD"  # Use SGD optimizer
)

Ultralytics 8.3.99 🚀 Python-3.8.13 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:1 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:2 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:3 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/home/mansi/Sumit Kumar, Satyendra/Eyes Final fgs/aptos org/data.yaml, epochs=100, time=None, patience=100, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=[0, 1, 2, 3], workers=16, project=None, name=aptos org, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, s

train: Scanning /home/mansi/Sumit Kumar, Satyendra/Eyes Final fgs/aptos org/train/labels... 2825 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2825/2825 [00:02<00:00, 1245.74it/s]


train: New cache created: /home/mansi/Sumit Kumar, Satyendra/Eyes Final fgs/aptos org/train/labels.cache


val: Scanning /home/mansi/Sumit Kumar, Satyendra/Eyes Final fgs/aptos org/valid/labels... 531 images, 0 backgrounds, 0 corrupt: 100%|██████████| 531/531 [00:00<00:00, 920.79it/s] 


val: New cache created: /home/mansi/Sumit Kumar, Satyendra/Eyes Final fgs/aptos org/valid/labels.cache
Plotting labels to runs/detect/aptos org/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001111, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 64 dataloader workers
Logging results to runs/detect/aptos org
Starting training for 100 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        531        531      0.697      0.757      0.769      0.769

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


    100/100      2.47G    0.01996     0.1719     0.8562          3        640: 100%|██████████| 45/45 [01:26<00:00,  1.93s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:12<00:00,  1.32it/s]


                   all        531        531      0.682      0.769      0.769      0.769

100 epochs completed in 2.474 hours.
Optimizer stripped from runs/detect/aptos org/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/aptos org/weights/best.pt, 6.3MB

Validating runs/detect/aptos org/weights/best.pt...
Ultralytics 8.3.99 🚀 Python-3.8.13 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:1 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:2 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:3 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
Model summary (fused): 72 layers, 3,006,623 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 17/17 [00:08<00:00,  1.93it/s]


                   all        531        531      0.698      0.757      0.769      0.769
                     0        277        277      0.982      0.996      0.995      0.995
                     1         46         46      0.614      0.609      0.717      0.717
                     2        136        136      0.702      0.882      0.837      0.837
                     3         30         30      0.581        0.7      0.669      0.669
                     4         42         42      0.607      0.595      0.626      0.626
Speed: 0.2ms preprocess, 1.5ms inference, 0.0ms loss, 2.7ms postprocess per image
Results saved to runs/detect/aptos org


In [5]:
# Validation
print("Running validation...")
metrics = model.val()
print("Validation Results:", metrics)

Running validation...
Ultralytics 8.3.99 🚀 Python-3.8.13 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:1 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:2 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)
                                                      CUDA:3 (NVIDIA GeForce RTX 2080 Ti, 11012MiB)


val: Scanning /home/mansi/Sumit Kumar, Satyendra/Eyes Final fgs/aptos org/valid/labels.cache... 531 images, 0 backgrounds, 0 corrupt: 100%|██████████| 531/5
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:16<00:00,  1.81s/it]


                   all        531        531      0.701      0.757      0.766      0.766
                     0        277        277      0.982      0.996      0.995      0.995
                     1         46         46      0.659      0.652      0.711      0.711
                     2        136        136      0.696      0.876      0.834      0.834
                     3         30         30      0.553      0.667      0.663      0.663
                     4         42         42      0.613      0.595      0.626      0.626
Speed: 0.2ms preprocess, 2.4ms inference, 0.0ms loss, 4.5ms postprocess per image
Results saved to runs/detect/aptos org3
Validation Results: ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ff5775e3eb0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Conf

In [ ]:
# Testing
print("Running testing...")
test_results = model.val(split="test")
print("Test Results:", test_results)